#### Case Study #1: Claim Resubmission Ingestion Pipeline

- Author: Daniel Dabiri
- Date: 2025
- Description: Robust pipeline for ingesting, normalizing, and analyzing insurance claim data from multiple EMR systems claims eligible for resubmission


In [2]:
%pip install pandas 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 1.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 2.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# IMPORTS AND SETUP
import json
import csv
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass, asdict
from enum import Enum
import pandas as pd
from pathlib import Path
import sys
from abc import ABC, abstractmethod


In [4]:
# Configure logging with detailed format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('claim_pipeline.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

##### ============================================================================

##### CONFIGURATION AND CONSTANTS

##### ============================================================================


In [5]:
class Config:
    """Central configuration for the pipeline"""
    REFERENCE_DATE = datetime(2025, 7, 30)  # Today's date for eligibility calculation
    DAYS_THRESHOLD = 7  # Claims must be older than this many days
    
    # Denial reason categories
    RETRYABLE_REASONS = {
        "missing modifier",
        "incorrect npi",
        "prior auth required"
    }
    
    NON_RETRYABLE_REASONS = {
        "authorization expired",
        "incorrect provider type"
    }
    
    # Ambiguous reasons that need classification
    AMBIGUOUS_MAPPING = {
        "incorrect procedure": False,  # Not retryable based on business logic
        "form incomplete": True,        # Retryable - can complete form
        "not billable": False,          # Not retryable
        None: False,                    # Null reasons not retryable by default
        "": False                       # Empty strings not retryable
    }

class ClaimStatus(Enum):
    """Enum for claim statuses"""
    APPROVED = "approved"
    DENIED = "denied"

##### ============================================================================

##### DATA MODEL

##### ============================================================================


In [6]:
@dataclass
class UnifiedClaim:
    """
    Unified schema for claims from all sources.
    This ensures consistent data structure across different EMR systems.
    """
    claim_id: str
    patient_id: Optional[str]
    procedure_code: str
    denial_reason: Optional[str]
    status: str
    submitted_at: str  # ISO format date
    source_system: str
    
    def to_dict(self) -> Dict:
        """Convert to dictionary for JSON serialization"""
        return asdict(self)
    
    def is_valid_for_resubmission(self) -> bool:
        """Check if claim has minimum required fields for resubmission"""
        return bool(self.claim_id and self.patient_id and self.status)

@dataclass
class ResubmissionCandidate:
    """Model for claims eligible for resubmission"""
    claim_id: str
    resubmission_reason: str
    source_system: str
    recommended_changes: str
    
    def to_dict(self) -> Dict:
        """Convert to dictionary for JSON serialization"""
        return asdict(self)

##### ============================================================================

##### DATA SOURCE HANDLERS

##### ============================================================================


In [7]:
class DataSourceHandler(ABC):
    """Abstract base class for data source handlers"""
    
    @abstractmethod
    def load_data(self, filepath: str) -> List[Dict]:
        """Load data from source file"""
        pass
    @abstractmethod
    def normalize(self, data: List[Dict]) -> List[UnifiedClaim]:
        """Normalize data to unified schema"""
        pass

class EMRAlphaHandler(DataSourceHandler):
    """Handler for EMR Alpha CSV data source"""
    
    def load_data(self, filepath: str) -> List[Dict]:
        """Load CSV data with robust error handling"""
        logger.info(f"Loading EMR Alpha data from {filepath}")
        data = []
        
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                for row_num, row in enumerate(reader, start=2):  # Start at 2 (header is line 1)
                    try:
                        data.append(row)
                    except Exception as e:
                        logger.warning(f"Error processing row {row_num}: {e}")
                        continue
                        
            logger.info(f"Successfully loaded {len(data)} records from EMR Alpha")
            return data
            
        except FileNotFoundError:
            logger.error(f"File not found: {filepath}")
            return []
        except Exception as e:
            logger.error(f"Error loading EMR Alpha data: {e}")
            return []
    
    def normalize(self, data: List[Dict]) -> List[UnifiedClaim]:
        """Normalize CSV data to unified schema"""
        logger.info("Normalizing EMR Alpha data")
        normalized = []
        
        for record in data:
            try:
                # Handle missing or empty patient_id
                patient_id = record.get('patient_id', '').strip()
                patient_id = patient_id if patient_id else None
                
                # Normalize denial reason (lowercase, handle 'None' string)
                denial_reason = record.get('denial_reason', '').strip()
                if denial_reason.lower() == 'none':
                    denial_reason = None
                elif denial_reason:
                    denial_reason = denial_reason.lower()
                else:
                    denial_reason = None
                
                # Parse and normalize date
                submitted_date = self._parse_date(record.get('submitted_at', ''))
                
                claim = UnifiedClaim(
                    claim_id=record.get('claim_id', '').strip(),
                    patient_id=patient_id,
                    procedure_code=record.get('procedure_code', '').strip(),
                    denial_reason=denial_reason,
                    status=record.get('status', '').lower().strip(),
                    submitted_at=submitted_date,
                    source_system="alpha"
                )
                
                if claim.claim_id:  # Only add if claim has an ID
                    normalized.append(claim)
                else:
                    logger.warning(f"Skipping record with missing claim_id: {record}")
                    
            except Exception as e:
                logger.warning(f"Error normalizing record {record}: {e}")
                continue
        
        logger.info(f"Normalized {len(normalized)} EMR Alpha records")
        return normalized
    
    def _parse_date(self, date_str: str) -> str:
        """Parse date to ISO format"""
        if not date_str:
            return ""
        
        try:
            # Try parsing YYYY-MM-DD format
            dt = datetime.strptime(date_str, "%Y-%m-%d")
            return dt.isoformat()
        except:
            try:
                # Try other common formats
                dt = datetime.strptime(date_str, "%m/%d/%Y")
                return dt.isoformat()
            except:
                logger.warning(f"Could not parse date: {date_str}")
                return date_str

class EMRBetaHandler(DataSourceHandler):
    """Handler for EMR Beta JSON data source"""
    
    def load_data(self, filepath: str) -> List[Dict]:
        """Load JSON data with robust error handling"""
        logger.info(f"Loading EMR Beta data from {filepath}")
        
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
            if not isinstance(data, list):
                logger.error("EMR Beta data is not a list")
                return []
                
            logger.info(f"Successfully loaded {len(data)} records from EMR Beta")
            return data
            
        except FileNotFoundError:
            logger.error(f"File not found: {filepath}")
            return []
        except json.JSONDecodeError as e:
            logger.error(f"Invalid JSON in EMR Beta file: {e}")
            return []
        except Exception as e:
            logger.error(f"Error loading EMR Beta data: {e}")
            return []
    
    def normalize(self, data: List[Dict]) -> List[UnifiedClaim]:
        """Normalize JSON data to unified schema"""
        logger.info("Normalizing EMR Beta data")
        normalized = []
        
        for record in data:
            try:
                # Map Beta field names to unified schema
                patient_id = record.get('member')
                if patient_id == "" or patient_id is None:
                    patient_id = None
                
                # Normalize denial reason
                denial_reason = record.get('error_msg')
                if denial_reason:
                    denial_reason = denial_reason.lower().strip()
                
                # Parse and normalize date (already in ISO format)
                submitted_date = record.get('date', '')
                if 'T' in submitted_date:
                    # Already in ISO format, just ensure it's complete
                    submitted_date = submitted_date
                
                claim = UnifiedClaim(
                    claim_id=record.get('id', '').strip(),
                    patient_id=patient_id,
                    procedure_code=record.get('code', '').strip(),
                    denial_reason=denial_reason,
                    status=record.get('status', '').lower().strip(),
                    submitted_at=submitted_date,
                    source_system="beta"
                )
                
                if claim.claim_id:  # Only add if claim has an ID
                    normalized.append(claim)
                else:
                    logger.warning(f"Skipping record with missing claim_id: {record}")
                    
            except Exception as e:
                logger.warning(f"Error normalizing record {record}: {e}")
                continue
        
        logger.info(f"Normalized {len(normalized)} EMR Beta records")
        return normalized

##### ============================================================================

##### DENIAL REASON CLASSIFIER

##### ============================================================================


In [8]:
class DenialReasonClassifier:
    """
    Classifier for determining if denial reasons are retryable.
    This simulates an LLM classifier with rule-based logic and heuristics.
    """
    
    def __init__(self):
        self.classification_cache = {}
        logger.info("Initialized DenialReasonClassifier")
    
    def is_retryable(self, denial_reason: Optional[str]) -> Tuple[bool, str]:
        """
        Determine if a denial reason is retryable.
        Returns: (is_retryable, classification_method)
        """
        if denial_reason is None or denial_reason == "":
            return False, "null_or_empty"
        
        # Normalize the reason
        normalized_reason = denial_reason.lower().strip()
        
        # Check cache first
        if normalized_reason in self.classification_cache:
            return self.classification_cache[normalized_reason], "cached"
        
        # Check known retryable reasons
        if normalized_reason in Config.RETRYABLE_REASONS:
            result = (True, "known_retryable")
        # Check known non-retryable reasons
        elif normalized_reason in Config.NON_RETRYABLE_REASONS:
            result = (False, "known_non_retryable")
        # Check ambiguous mapping
        elif normalized_reason in Config.AMBIGUOUS_MAPPING:
            is_retry = Config.AMBIGUOUS_MAPPING[normalized_reason]
            result = (is_retry, "ambiguous_mapped")
        else:
            # Use heuristic classifier for unknown reasons
            result = (self._heuristic_classify(normalized_reason), "heuristic")
        
        # Cache the result
        self.classification_cache[normalized_reason] = result[0]
        logger.debug(f"Classified '{denial_reason}' as retryable={result[0]} using {result[1]}")
        
        return result
    
    def _heuristic_classify(self, reason: str) -> bool:
        """
        Heuristic classification for unknown denial reasons.
        This simulates an LLM decision based on keywords and patterns.
        """
        # Keywords that suggest retryable issues
        retryable_keywords = [
            'missing', 'incomplete', 'required', 'update', 'resubmit',
            'incorrect format', 'needs', 'add', 'provide'
        ]
        
        # Keywords that suggest non-retryable issues
        non_retryable_keywords = [
            'expired', 'invalid', 'not covered', 'excluded', 'terminated',
            'not eligible', 'duplicate', 'already processed'
        ]
        
        reason_lower = reason.lower()
        
        # Check for non-retryable keywords first (more definitive)
        for keyword in non_retryable_keywords:
            if keyword in reason_lower:
                return False
        
        # Check for retryable keywords
        for keyword in retryable_keywords:
            if keyword in reason_lower:
                return True
        
        # Default to non-retryable for unknown patterns
        return False
    
    def get_recommendation(self, denial_reason: str) -> str:
        """Get recommended changes for a denial reason"""
        if denial_reason is None:
            return "Review claim for completeness"
        
        reason_lower = denial_reason.lower()
        
        recommendations = {
            "missing modifier": "Add appropriate modifier codes to the claim",
            "incorrect npi": "Review NPI number and resubmit",
            "prior auth required": "Obtain prior authorization and resubmit",
            "form incomplete": "Complete all required form fields",
            "incorrect procedure": "Verify procedure code accuracy",
        }
        
        return recommendations.get(reason_lower, f"Address issue: {denial_reason}")

##### ============================================================================

##### RESUBMISSION ELIGIBILITY ENGINE

##### ============================================================================


In [9]:
class ResubmissionEligibilityEngine:
    """Engine for determining claim resubmission eligibility"""
    
    def __init__(self, classifier: DenialReasonClassifier):
        self.classifier = classifier
        self.stats = {
            'total_processed': 0,
            'denied_claims': 0,
            'missing_patient_id': 0,
            'too_recent': 0,
            'non_retryable_reason': 0,
            'eligible': 0
        }
        logger.info("Initialized ResubmissionEligibilityEngine")
    
    def evaluate_claims(self, claims: List[UnifiedClaim]) -> List[ResubmissionCandidate]:
        """Evaluate claims for resubmission eligibility"""
        logger.info(f"Evaluating {len(claims)} claims for resubmission eligibility")
        candidates = []
        
        for claim in claims:
            self.stats['total_processed'] += 1
            
            # Check eligibility criteria
            if self._is_eligible(claim):
                candidate = ResubmissionCandidate(
                    claim_id=claim.claim_id,
                    resubmission_reason=claim.denial_reason or "Unknown",
                    source_system=claim.source_system,
                    recommended_changes=self.classifier.get_recommendation(claim.denial_reason or "")
                )
                candidates.append(candidate)
                self.stats['eligible'] += 1
                logger.debug(f"Claim {claim.claim_id} is eligible for resubmission")
        
        logger.info(f"Found {len(candidates)} eligible claims for resubmission")
        return candidates
    
    def _is_eligible(self, claim: UnifiedClaim) -> bool:
        """
        Check if a claim is eligible for resubmission.
        
        Criteria:
        1. Status is denied
        2. Patient ID is not null
        3. Claim was submitted more than 7 days ago
        4. Denial reason is retryable
        """
        # Criterion 1: Status must be denied
        if claim.status != ClaimStatus.DENIED.value:
            return False
        
        self.stats['denied_claims'] += 1
        
        # Criterion 2: Patient ID must not be null
        if not claim.patient_id:
            self.stats['missing_patient_id'] += 1
            logger.debug(f"Claim {claim.claim_id} missing patient_id")
            return False
        
        # Criterion 3: Claim must be older than threshold
        try:
            submitted_date = datetime.fromisoformat(claim.submitted_at.replace('Z', '+00:00'))
            days_old = (Config.REFERENCE_DATE - submitted_date.replace(tzinfo=None)).days
            
            if days_old <= Config.DAYS_THRESHOLD:
                self.stats['too_recent'] += 1
                logger.debug(f"Claim {claim.claim_id} too recent ({days_old} days old)")
                return False
                
        except Exception as e:
            logger.warning(f"Could not parse date for claim {claim.claim_id}: {e}")
            return False
        
        # Criterion 4: Denial reason must be retryable
        is_retryable, method = self.classifier.is_retryable(claim.denial_reason)
        if not is_retryable:
            self.stats['non_retryable_reason'] += 1
            logger.debug(f"Claim {claim.claim_id} has non-retryable reason: {claim.denial_reason}")
            return False
        
        return True
    
    def get_statistics(self) -> Dict:
        """Get processing statistics"""
        return self.stats.copy()


##### ============================================================================

##### MAIN PIPELINE ORCHESTRATOR

##### ============================================================================


In [10]:
class ClaimResubmissionPipeline:
    """Main pipeline orchestrator for claim resubmission processing"""
    
    def __init__(self):
        self.alpha_handler = EMRAlphaHandler()
        self.beta_handler = EMRBetaHandler()
        self.classifier = DenialReasonClassifier()
        self.eligibility_engine = ResubmissionEligibilityEngine(self.classifier)
        self.metrics = {
            'source_counts': {},
            'total_claims': 0,
            'eligible_claims': 0,
            'processing_errors': 0
        }
        logger.info("Initialized ClaimResubmissionPipeline")
    
    def run(self, alpha_file: str, beta_file: str, output_file: str = "resubmission_candidates.json"):
        """
        Execute the complete pipeline.
        
        Args:
            alpha_file: Path to EMR Alpha CSV file
            beta_file: Path to EMR Beta JSON file
            output_file: Path for output JSON file
        """
        logger.info("=" * 60)
        logger.info("Starting Claim Resubmission Pipeline")
        logger.info("=" * 60)
        
        try:
            # Step 1: Load and normalize data from all sources
            all_claims = self._load_and_normalize_data(alpha_file, beta_file)
            
            if not all_claims:
                logger.error("No claims loaded from any source")
                return
            
            # Step 2: Evaluate claims for resubmission eligibility
            candidates = self.eligibility_engine.evaluate_claims(all_claims)
            
            # Step 3: Save results
            self._save_results(candidates, output_file)
            
            # Step 4: Generate and display metrics
            self._generate_metrics(candidates)
            
            # Step 5: Create rejection log
            self._create_rejection_log(all_claims, candidates)
            
            logger.info("=" * 60)
            logger.info("Pipeline completed successfully")
            logger.info("=" * 60)
            
        except Exception as e:
            logger.error(f"Pipeline failed: {e}")
            self.metrics['processing_errors'] += 1
            raise
    
    def _load_and_normalize_data(self, alpha_file: str, beta_file: str) -> List[UnifiedClaim]:
        """Load and normalize data from all sources"""
        all_claims = []
        
        # Process EMR Alpha
        logger.info("Processing EMR Alpha source...")
        alpha_data = self.alpha_handler.load_data(alpha_file)
        alpha_claims = self.alpha_handler.normalize(alpha_data)
        all_claims.extend(alpha_claims)
        self.metrics['source_counts']['alpha'] = len(alpha_claims)
        
        # Process EMR Beta
        logger.info("Processing EMR Beta source...")
        beta_data = self.beta_handler.load_data(beta_file)
        beta_claims = self.beta_handler.normalize(beta_data)
        all_claims.extend(beta_claims)
        self.metrics['source_counts']['beta'] = len(beta_claims)
        
        self.metrics['total_claims'] = len(all_claims)
        logger.info(f"Total claims loaded: {len(all_claims)}")
        
        return all_claims
    
    def _save_results(self, candidates: List[ResubmissionCandidate], output_file: str):
        """Save resubmission candidates to JSON file"""
        logger.info(f"Saving {len(candidates)} candidates to {output_file}")
        
        try:
            output_data = [candidate.to_dict() for candidate in candidates]
            
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(output_data, f, indent=2, ensure_ascii=False)
            
            logger.info(f"Successfully saved results to {output_file}")
            
        except Exception as e:
            logger.error(f"Error saving results: {e}")
            raise
    
    def _generate_metrics(self, candidates: List[ResubmissionCandidate]):
        """Generate and display processing metrics"""
        self.metrics['eligible_claims'] = len(candidates)
        eligibility_stats = self.eligibility_engine.get_statistics()
        
        # Print comprehensive metrics
        print("\n" + "=" * 60)
        print("PIPELINE METRICS")
        print("=" * 60)
        print(f"\n📊 Data Sources:")
        for source, count in self.metrics['source_counts'].items():
            print(f"  - EMR {source.capitalize()}: {count} claims")
        
        print(f"\n📈 Processing Summary:")
        print(f"  - Total claims processed: {self.metrics['total_claims']}")
        print(f"  - Denied claims: {eligibility_stats['denied_claims']}")
        print(f"  - Eligible for resubmission: {self.metrics['eligible_claims']}")
        
        print(f"\n❌ Exclusion Reasons:")
        print(f"  - Missing patient ID: {eligibility_stats['missing_patient_id']}")
        print(f"  - Too recent (≤7 days): {eligibility_stats['too_recent']}")
        print(f"  - Non-retryable denial reason: {eligibility_stats['non_retryable_reason']}")
        
        if self.metrics['eligible_claims'] > 0:
            print(f"\n✅ Resubmission Rate: {self.metrics['eligible_claims']/self.metrics['total_claims']*100:.1f}%")
        
        # Save metrics to file
        with open('pipeline_metrics.json', 'w') as f:
            json.dump(self.metrics, f, indent=2)
        
        logger.info("Metrics saved to pipeline_metrics.json")
    
    def _create_rejection_log(self, all_claims: List[UnifiedClaim], candidates: List[ResubmissionCandidate]):
        """Create a log of rejected claims"""
        eligible_ids = {c.claim_id for c in candidates}
        rejected_claims = [
            claim.to_dict() for claim in all_claims 
            if claim.claim_id not in eligible_ids and claim.status == 'denied'
        ]
        
        if rejected_claims:
            with open('rejected_claims.json', 'w') as f:
                json.dump(rejected_claims, f, indent=2)
            logger.info(f"Saved {len(rejected_claims)} rejected claims to rejected_claims.json")

##### ============================================================================

##### SAMPLE DATA CREATION (for testing)

##### ============================================================================


In [11]:
def create_sample_data():
    """Create sample data files for testing the pipeline"""
    
    # Create EMR Alpha CSV
    alpha_data = """claim_id,patient_id,procedure_code,denial_reason,submitted_at,status
A123,P001,99213,Missing modifier,2025-07-01,denied
A124,P002,99214,Incorrect NPI,2025-07-10,denied
A125,,99215,Authorization expired,2025-07-05,denied
A126,P003,99381,None,2025-07-15,approved
A127,P004,99401,Prior auth required,2025-07-20,denied"""
    
    with open('emr_alpha.csv', 'w') as f:
        f.write(alpha_data)
    
    # Create EMR Beta JSON
    beta_data = [
        {
            "id": "B987",
            "member": "P010",
            "code": "99213",
            "error_msg": "Incorrect provider type",
            "date": "2025-07-03T00:00:00",
            "status": "denied"
        },
        {
            "id": "B988",
            "member": "P011",
            "code": "99214",
            "error_msg": "Missing modifier",
            "date": "2025-07-09T00:00:00",
            "status": "denied"
        },
        {
            "id": "B989",
            "member": "P012",
            "code": "99215",
            "error_msg": None,
            "date": "2025-07-10T00:00:00",
            "status": "approved"
        },
        {
            "id": "B990",
            "member": None,
            "code": "99401",
            "error_msg": "incorrect procedure",
            "date": "2025-07-01T00:00:00",
            "status": "denied"
        }
    ]
    
    with open('emr_beta.json', 'w') as f:
        json.dump(beta_data, f, indent=2)
    
    print("Sample data files created: emr_alpha.csv and emr_beta.json")

##### ============================================================================

##### MAIN EXECUTION

##### ============================================================================


In [12]:
if __name__ == "__main__":
    # Create sample data files
    create_sample_data()
    
    # Initialize and run the pipeline
    pipeline = ClaimResubmissionPipeline()
    
    # Run with sample data
    pipeline.run(
        alpha_file='emr_alpha.csv',
        beta_file='emr_beta.json',
        output_file='resubmission_candidates.json'
    )
    
    # Display the output
    print("\n" + "=" * 60)
    print("RESUBMISSION CANDIDATES")
    print("=" * 60)
    
    with open('resubmission_candidates.json', 'r') as f:
        candidates = json.load(f)
        print(json.dumps(candidates, indent=2))
    
    print(f"\n✅ Pipeline execution completed!")
    print(f"📁 Output files created:")
    print(f"  - resubmission_candidates.json (eligible claims)")
    print(f"  - rejected_claims.json (non-eligible denied claims)")
    print(f"  - pipeline_metrics.json (processing statistics)")
    print(f"  - claim_pipeline.log (detailed logs)")

Sample data files created: emr_alpha.csv and emr_beta.json
2025-08-18 21:03:55,350 - __main__ - INFO - Initialized DenialReasonClassifier
2025-08-18 21:03:55,358 - __main__ - INFO - Initialized ResubmissionEligibilityEngine
2025-08-18 21:03:55,360 - __main__ - INFO - Initialized ClaimResubmissionPipeline
2025-08-18 21:03:55,363 - __main__ - INFO - ============================================================
2025-08-18 21:03:55,364 - __main__ - INFO - Starting Claim Resubmission Pipeline
2025-08-18 21:03:55,365 - __main__ - INFO - ============================================================
2025-08-18 21:03:55,367 - __main__ - INFO - Processing EMR Alpha source...
2025-08-18 21:03:55,371 - __main__ - INFO - Loading EMR Alpha data from emr_alpha.csv
2025-08-18 21:03:55,374 - __main__ - INFO - Successfully loaded 5 records from EMR Alpha
2025-08-18 21:03:55,375 - __main__ - INFO - Normalizing EMR Alpha data
2025-08-18 21:03:55,378 - __main__ - INFO - Normalized 5 EMR Alpha records
2025-08